# Implementação

In [2]:
def ODEsys(t,y,params):
    a,c,s = params
    A = y
    return[(1 - A)*c*s*(A)**a + A*c*(1-s)*(1-A)**a]

In [9]:
T=ode_solver()
T.algorithm="rk8pd"
T.function=ODEsys

In [10]:
T.solution[:10]

[]